# Подгружаем необходимые библиотеки

In [79]:
import numpy as np
import random
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go
import requests

# Задание 1
Тяжело понять, что необходимо отфильтровать, а что оставить. В задании сказано *отфильтровать* числа, которые делятся без остатка. Кажется, что это значит, что такие числа нужно убрать. На всякий случай прикладываю 2 варианта

In [80]:
base_arr = np.array([i for i in range(10, 101, 1)]) # создаем array с числами
random_arr = np.array([random.choice(base_arr) for _ in range(50)]) # случайно выбираем 50 чисел из массива

print('Среднее арифметическое массива:', random_arr.mean())
print('Медиана массива:', np.median(random_arr))
print('Стандартное отклонение:', np.std(random_arr))

divided_arr = random_arr/3 # делим на 3
print('Массив без фильтра: ', divided_arr)
divided_arr_filtred = np.extract(divmod(divided_arr, 1)[1]!=0, divided_arr) # делаем фильтр, при котором деление на 1 создает остаток (т. е. остаток был изначально)
print('Деление на 3, с фильтром на числа без остатка', divided_arr_filtred)
divided_arr_filtred_2 = np.extract(divmod(divided_arr, 1)[1]==0, divided_arr)# делаем фильтр, при котором деление на 1 создает остаток (т. е. остаток был изначально)
print('Деление на 3, с фильтром на числа c остатком', divided_arr_filtred_2)

Среднее арифметическое массива: 51.78
Медиана массива: 54.0
Стандартное отклонение: 28.002349901392204
Массив без фильтра:  [31.          6.33333333 24.66666667  9.          4.66666667  9.66666667
 25.66666667  4.         23.66666667 23.66666667  9.66666667  5.
  9.          3.33333333  6.33333333 27.33333333 16.66666667  4.
 23.66666667 26.66666667 25.66666667 31.         10.         26.
 22.33333333 26.33333333  7.         10.33333333 18.          7.
  7.33333333  6.66666667 33.33333333 18.         28.          6.33333333
 14.33333333 21.33333333  5.66666667 17.33333333 28.33333333 28.
 18.66666667 25.         17.33333333  6.66666667 28.66666667 21.66666667
 30.66666667 22.        ]
Деление на 3, с фильтром на числа без остатка [ 6.33333333 24.66666667  4.66666667  9.66666667 25.66666667 23.66666667
 23.66666667  9.66666667  3.33333333  6.33333333 27.33333333 16.66666667
 23.66666667 26.66666667 25.66666667 22.33333333 26.33333333 10.33333333
  7.33333333  6.66666667 33.33333333  6.3

# Задание 2

In [ ]:
df = pd.read_csv('./data/sales_data_large.csv')
print('Количество записей:', df.shape[0])
print('Регионы в файле данных:', df['region'].unique())
print('Продукты в файле данных:', df['product'].unique())

region_df = df[['region', 'sales']].groupby('region').sum() # группировка по региону с агрегацией по сумме
print('Общие продажи по регионам')
print(region_df)

product_df = df[['product', 'sales']].groupby('product').mean() # группировка по продукту с агрегацией по среднему
print('Средний объем продаж для каждого продукта')
print(product_df)

pivot_table = pd.pivot_table(df, values='sales', index='region',
                         columns='product', aggfunc="mean") # сводная таблица по продукту и региону с агрегацией по среднему
print('Сводная таблица, показывающая средние продажи по регионам и продуктам')
print(pivot_table)

Количество записей: 500
Регионы в файле данных: ['West' 'South' 'East' 'North']
Продукты в файле данных: ['Widget C' 'Widget B' 'Widget A']
Общие продажи по регионам
        sales
region       
East    29244
North   39605
South   33023
West    35171
Средний объем продаж для каждого продукта
               sales
product             
Widget A  275.424242
Widget B  268.331361
Widget C  278.614458
Сводная таблица, показывающая средние продажи по регионам и продуктам
product    Widget A    Widget B    Widget C
region                                     
East     291.464286  263.116279  271.361111
North    289.145833  238.477273  304.660000
South    277.380952  282.833333  263.722222
West     250.106383  291.550000  267.136364


# Задание 3
1) Для дат взято среднее значение (по рекомендации ментора);
2) Среднее округлено до целого по правилам математического округления.

In [85]:
df = pd.read_csv('./data/sales_cleaning_large.csv')
df['date']=pd.to_datetime(df['date'])
df['date'] = df['date'].fillna(df['date'].mean()) # заполняем пропущенные даты
df = df.drop(df.loc[df['sales']<0].index) # удаляем отрицательные значения (т. к. сказано, что можно удалять)
df['sales'] = df['sales'].fillna(round(df['sales'].mean(), 0)) # заполняем пропущенные значения в sales

print('Предобработанный DataFrame:')
print(df)
# строим график продаж по датам
df_sales = df[['date', 'sales']].groupby('date').sum() # формируем df с информацией о продажах в рамке даты
df_sales.sort_index() # убеждаемся, что даты упорядочены
fig = go.Figure(data=[
    go.Scatter(
        x=df_sales.index,
        y=df_sales['sales']
    )
], layout={
    'title': 'Sales plot by date'
})
fig.show()

Предобработанный DataFrame:
                             date  sales region
1   2024-12-01 00:00:00.000000000  400.0   East
3   2024-12-01 00:00:00.000000000  350.0   West
4   2024-12-02 00:00:00.000000000  400.0  South
5   2024-12-02 00:00:00.000000000   50.0  North
6   2024-12-02 00:00:00.000000000  250.0   West
..                            ...    ...    ...
189 2024-12-31 00:00:00.000000000  400.0   West
190 2024-12-15 19:27:09.473684224  150.0  South
192 2024-12-15 19:27:09.473684224   50.0   West
193 2024-12-15 19:27:09.473684224  350.0  South
195 2024-12-15 19:27:09.473684224  400.0   West

[111 rows x 3 columns]


# Задание 4

In [83]:
req = requests.get('https://jsonplaceholder.typicode.com/users')
json = req.json() # получаем ответ в виде JSON

columns = ['id', 'name', 'email', 'company_name'] # создаем колонки
df_arr = []
for i in json:
    df_arr.append([i['id'], i['name'], i['email'], i['company']['name']]) # заполняем данные из json-а
df = pd.DataFrame(df_arr, columns=columns) # формируем DF
print('Таблица, содержащая пользователей, которые работают в компании, в которой содержится слово "Group"')
print(df.loc[df['company_name'].str.find('Group')!=-1]) # фильтрация

Таблица, содержащая пользователей, которые работают в компании, в которой содержится слово "Group"
   id                      name                   email     company_name
6   7           Kurtis Weissnat  Telly.Hoeger@billy.biz      Johns Group
7   8  Nicholas Runolfsdottir V    Sherwood@rosamond.me  Abernathy Group
